In [2]:
!pip install psycopg2

     ---------------------------------------- 2.7/2.7 MB 8.2 MB/s eta 0:00:00


In [3]:
"""
Example 1: Basic Sequential Load
--------------------------------
Demonstrates inserting records into CockroachDB one by one.
"""

import psycopg2
import logging

# Database setup (replace with your details)
DB_CONFIG = {
    "host": "localhost",
    "port": 26257,
    "user": "root",
    "dbname": "supply_chain"
}

# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
logger = logging.getLogger("cockroach_basic")

def insert_data(records):
    """Sequential insertion."""
    conn = psycopg2.connect(**DB_CONFIG)
    cursor = conn.cursor()
    for rec in records:
        cursor.execute(
            "INSERT INTO dashboard_metrics (region, orders, revenue) VALUES (%s, %s, %s)",
            rec
        )
    conn.commit()
    conn.close()
    logger.info("✅ Sequential insert completed successfully.")

if __name__ == "__main__":
    sample_records = [
        ("APAC", 500, 25000),
        ("EMEA", 800, 40000),
        ("US", 700, 35000)
    ]
    insert_data(sample_records)


OperationalError: connection to server at "localhost" (::1), port 26257 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 26257 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?


In [ ]:
"""
Example 2: Parallel Load into CockroachDB
-----------------------------------------
Loads data into CockroachDB in parallel using ThreadPoolExecutor.
"""

import psycopg2
from concurrent.futures import ThreadPoolExecutor
import logging
import random
import time

# Configuration
DB_CONFIG = {
    "host": "localhost",
    "port": 26257,
    "user": "root",
    "dbname": "supply_chain"
}

# Logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
logger = logging.getLogger("cockroach_parallel")

def insert_partition(partition_data):
    """Insert partition data into CockroachDB."""
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        cursor = conn.cursor()
        for rec in partition_data:
            cursor.execute(
                "INSERT INTO dashboard_metrics (region, orders, revenue) VALUES (%s, %s, %s)",
                rec
            )
        conn.commit()
        conn.close()
        logger.info(f"✅ Loaded partition with {len(partition_data)} records.")
    except Exception as e:
        logger.error(f"❌ Error loading partition: {e}")

def simulate_data():
    """Simulate partitioned data for parallel loading."""
    regions = ["APAC", "EMEA", "US", "LATAM"]
    return [
        [(r, random.randint(100, 1000), random.randint(10000, 90000)) for _ in range(10)]
        for r in regions
    ]

if __name__ == "__main__":
    partitions = simulate_data()
    logger.info("🚀 Starting parallel load into CockroachDB...")

    with ThreadPoolExecutor(max_workers=4) as executor:
        executor.map(insert_partition, partitions)

    logger.info("🏁 All partitions loaded successfully.")
